In [402]:
 import math 
from sympy import*
from sympy.abc import*

def F_z(y):
    H = 0.012
    z = y / (math.sqrt(2) * H)
    F_z = (2 / math.sqrt(math.pi)) * (integrate(exp(-(x ** 2)), (x, 0, z)).evalf())
    return F_z

def dolya(x1, x2, y1, y2):
    ras = (1/4) * (F_z(x2) - F_z(x1)) * (F_z(y2) - F_z(y1))
    return ras

def dolya_prist(x1, x2, y1):
    ras = (1/4) * (F_z(x2) - F_z(x1)) * (1 - F_z(y1))
    return ras
def okrug(x):
    x = x * 10000
    ost = x % 1
    if ost >= 0.5:
        x = (x // 1) + 1
    else: 
        x = x // 1
    if x == 0:
        x = 1
    x /= 10000
    return x

def nax_fi(alfa):
    fi = 0.1
    t = math.tan(alfa * math.pi / 180)
    A_fi = ((1 - fi) * (2 ** 0.5)) / (fi ** 1.5)
    mu_fi = fi * ((fi / (2 - fi)) ** 0.5)
    t1 = (2 * A_fi * mu_fi) / ((((1 + ((1 - fi) ** 0.5)
                                 ) ** 2) - (4 * ((mu_fi * A_fi) ** 2))) ** 0.5)
    ta = (t * 1000) // 1
    ta1 = (t1 * 1000) // 1
    while ta != ta1:
        t = math.tan(alfa * math.pi / 180)
        A_fi = ((1 - fi) * (2 ** 0.5)) / (fi ** 1.5)
        mu_fi = fi * ((fi / (2 - fi)) ** 0.5)
        t1 = (2 * A_fi * mu_fi) / ((((1 + ((1 - fi) ** 0.5)) ** 2) -
                                    (4 * ((mu_fi * A_fi) ** 2))) ** 0.5)
        fi += 0.00001
        ta = (t * 1000) // 1
        ta1 = (t1 * 1000) // 1
    return fi

## Исходные данные

$P_п = 100 кН;~ p_к = 10 МПа;~p_a = 0.005 МПа \\Компоненты~топлива ~~~~ O_2 + H_2$

In [403]:
P_pust = 100000
p_k = 10 ** 7
p_a = 0.005 * 10**6

## Определение основных парраметров

### в ядре потока

$\alpha_{опт.я} = 0.73;~ K_0 = 7.937;~ K_m = 5.794;~ I_уп = 4624.2~\frac{м}{c}; \\ T_я = 3696.47K; R= 630 \frac{Дж}{кг \cdot К};~ k= 1.272$

In [404]:
alfa_optim_yadr = 0.73
K_0 = 7.937
K_m = alfa_optim_yadr * K_0
I_u_yadr = 4624.2
T_yadr = 3696.47
R = 630
k = 1.272

### В пристенеке 

$ T_{прист} = 2000 К;~ \alpha_{прист} = 0.29;~ K_{mприст} = 2.302;~ I_{уприст} = 4314.7$

In [405]:
T_prist = 2000
alfa_prist = 0.29
K_m_prist = alfa_prist * K_0
I_u_prist = 4314.7

### Параметры с учетом избытка горючего в пристенке
$ I_{уд} = 4608.7~ \frac{м}{с};~~ \dot m_{\Sigma} = 21.7~ \frac{кг}{с}$

In [406]:
I_ud = I_u_yadr - (0.05 * (I_u_yadr - I_u_prist))
m_sum = P_pust / I_ud

#### Расходы в пристенке 

$ \dot m_{прист} = 1.085~\frac{кг}{с};~ \dot m_{гприст} = 0.329~\frac{кг}{с}~ \dot m_{окприст} = 0.756~\frac{кг}{с}$

In [407]:
m_prist = m_sum * 0.05
m_g_prist = m_prist / (K_m_prist + 1)
m_ok_prist = m_prist - m_g_prist

#### Расходы в ядре 

$ \dot m_{я} = 20.61 \frac{кг}{с};~ \dot m_{гя} = 3.034 \frac{кг}{с};~ \dot m_{окя} = 17.58 \frac{кг}{с}$

In [408]:
m_yadr = m_sum - m_prist
m_g_yadr = m_yadr / (K_m + 1)
m_ok_yadr = m_yadr - m_g_yadr

#### Расходный комплекс

$ A_k = 0.662;~~ \beta = 2304.65\frac{м}{с}$

In [409]:
A_k = (k ** 0.5) * ((2 / (k + 1)) ** ((k + 1)  / (2 * (k - 1))))
betta = ((T_yadr * R) ** 0.5) / A_k

#### Параметры критического сечения 

$F_{кр} = 0.005 м^2;~ D_{кр} = 0.0798 м$

In [410]:
F_kr = (m_sum * betta) / p_k
D_kr = ((4 * F_kr) / 3.14) ** 0.5

#### Параметры форсуночной головки
##### Находим радиус форсуночной головкии и задаемся шагом между форсунками

$F_г = 0.02 м^2;~ D_г = 0.16 м;~ R_г = 0.08м;~ H = 0.012м;~ D_ш = 0.129м;$

In [411]:
F_g = 4 * F_kr
D_g = ((4 * F_g) / 3.14) ** 0.5
R_g = D_g / 2
H = 0.012
D_sh = D_g - (2.5 * H)
D_sh

0.12962757350946136

##### Определяем радиус расположения пристеночных форсунок 

$R_{прист} = 0.0738 м;$

In [412]:
R_prist = (D_g - H) / 2
R_prist

0.07381378675473067

#### Задаемся количеством форсунок в ядре и пристенке 
$n_{гпр} = 108;~ n_{окпр} = 36;~ n_{гя} = 65;~ n_{окпр} = 138;~$

In [413]:
n_g_prist = 108
n_ok_prist = 36
n_g_yadr = 65
n_ok_yadr =138 

#### Определеим соответствующие расходы через форсунки 

$\dot m_{фгя} = 0.0467 \frac{кг}{с};~ \dot m_{фокя} = 0.127 \frac{кг}{с};~\dot m_{фгприст} = 0.003 \frac{кг}{с};~\dot m_{я} = 0.005 \frac{кг}{с};~$

In [414]:
m_fg_yadr = m_g_yadr / n_g_yadr
m_fok_yadr = m_ok_yadr / n_ok_yadr
m_fg_prist = m_g_prist / n_g_prist
m_fok_prist = m_ok_prist / n_ok_yadr

### Геометрический метод расчета соотношения компонентов в ядре потока

#### Для площадки с форсункой горючего по центру и форсунками окислителя по краям квадрата справедливо

$K_{km~геом} = 5.458$

In [415]:
n_o = 6 # Количество форсунок окислителя
m_otn_ok = 1 / 3 # Относительный расход каждой форсунки окислителя на площадку
n_g = 1 # Количество форсунок окислителя 
m_otn_g = 1 # Относительный расход каждой форсунки горючего на площадку
k_m_yadr_geom = (n_o * m_otn_ok * m_fok_yadr) / (n_g * m_otn_g * m_fg_yadr)

### Расчет соотношения компонентов методом Иевлева

#### Приняв что ддля каждого элемента принимабт участие форсунки, которые находятся на растояни 3-х шагов 

##### Площадка №1 (центральная)
$K_{m1} = 5.111$

In [416]:
XX1ok = [(-0.0065, 0.0065), (-0.0144, -0.00043), (-0.0065, 0.0065),
       (-0.02035, -0.00735), (-0.001342, -0.00042), (-0.02035, -0.00735)]
YY1ok = [(-0.0145, -0.0015), (-0.0105, 0.0025), (-0.0225, -0.0095), 
       (-0.0145, -0.0015), (-0.0265, -0.0135), (-0.0225, -0.0095)]
d_sum = 0
for i1 in range(len(XX1ok)):
    d = dolya(XX1ok[i1][0], XX1ok[i1][1], YY1ok[i1][0], YY1ok[i1][1])
    d_sum += d
m_ok1 = 6* m_fok_yadr * d_sum

XX1g = [(-0.0065, 0.0065), (-0.0144, -0.00043)]
YY1g = [(-0.0065, 0.0065), (-0.0185, -0.0055)]
d_sum = 0
for i1 in range(len(XX1g)):
    d = dolya(XX1g[i1][0], XX1g[i1][1], YY1g[i1][0], YY1g[i1][1])
    d_sum += d
m_g1 = 6* m_fg_yadr * d_sum
k_m1 = m_ok1 / m_g1

#### Площадки №2, 3 по радиуссу:
#### Распределение аналогично распределению площади №1, ввиду той же геометрии расположения форсунок в рассматриваемой зоне 3Н

#### Площадка 4 по радиусу:
$K_{m4} = 5.021$

In [417]:
XX4ok = [(-0.0065, 0.0065), (-0.0144, -0.00043), (-0.0065, 0.0065),
       (-0.02035, -0.00735), (-0.001342, -0.00042), (-0.02035, -0.00735)]
YY4ok = [(-0.0145, -0.0015), (-0.0105, 0.0025), (-0.0225, -0.0095), 
       (-0.0145, -0.0015), (-0.0265, -0.0135), (-0.0225, -0.0095)]
d_sum = 0
for i4 in range(len(XX4ok)):
    d = dolya(XX4ok[i4][0], XX4ok[i4][1], YY4ok[i4][0], YY4ok[i4][1])
    if i4 >= 4 :
        d_sum += 5 * d
    else:
        d_sum += 6 * d
m_ok4 = m_fok_yadr * d_sum

XX4g = [(-0.0065, 0.0065), (-0.0144, -0.00043)]
YY4g = [(-0.0065, 0.0065), (-0.0185, -0.0055)]
d_sum = 0
for i4 in range(len(XX4g)):
    d = dolya(XX4g[i4][0], XX4g[i4][1], YY4g[i4][0], YY4g[i4][1])
    d_sum += 6 * d

m_g4 = m_fg_yadr * d_sum
k_m4 = m_ok4 / m_g4 

#### Площадка №5 по радиусу:
$K_{m5} = 3.35$

In [418]:
XX5ok_yadr = [(-0.0065, 0.0065), (-0.0144, -0.00043), (-0.0065, 0.0065),
       (-0.02035, -0.00735), (-0.001342, -0.00042), (-0.02035, -0.00735),
             (-0.0065, 0.0065), (-0.0144, -0.00043), (-0.0065, 0.0065),
       (-0.02035, -0.00735), (-0.001342, -0.00042), (-0.02035, -0.00735), 
             (-0.001342, -0.00042)]
YY5ok_yadr = [(-0.0145, -0.0015), (-0.0105, 0.0025), (-0.0225, -0.0095), 
       (-0.0145, -0.0015), (-0.0265, -0.0135), (-0.0225, -0.0095),
             (-0.0145, -0.0015), (-0.0105, 0.0025), (-0.0225, -0.0095), 
       (-0.0145, -0.0015), (-0.0265, -0.0135), (-0.0225, -0.0095),
             (-0.0265, -0.0135)]
d_sum_yadr = 0

for i5 in range(len(XX5ok_yadr)):
    d = dolya(XX5ok_yadr[i5][0], XX5ok_yadr[i5][1], YY5ok_yadr[i5][0], YY5ok_yadr[i5][1])
    d_sum_yadr += d
m_ok5_yadr = m_fok_yadr * d_sum_yadr

In [419]:
XX5ok_prist = [(-0.0193, -0.0063)]
YY5ok_prist = [(-0.0125, 0.0005)]
d_sum_prist = 0

for i5 in range(len(XX5ok_prist)):
    d = dolya(XX5ok_prist[i5][0], XX5ok_prist[i5][1], YY5ok_prist[i5][0], YY5ok_prist[i5][1])
    d_sum_prist += 2 * d
m_ok5_prist = m_fok_prist * d_sum_prist
m_ok5 = m_ok5_prist + m_ok5_yadr

In [420]:
XX5g_yadr = [(-0.0065, 0.0065), (-0.0144, -0.00043)]
YY5g_yadr = [(-0.0065, 0.0065), (-0.0185, -0.0055)]
d_sum_yadr = 0

for i5 in range(len(XX5g_yadr)):
    d = dolya(XX5g_yadr[i5][0], XX5g_yadr[i5][1], YY5g_yadr[i5][0], YY5g_yadr[i5][1])
    d_sum_yadr += 3*d
m_g5_yadr = m_fg_yadr * d_sum_yadr

In [421]:
XX5g_prist = [(-0.0185, -0.0055), (-0.0196, -0.0066),
             (-0.0248, -0.0118), (-0.0121, -0.0251)]
YY5g_prist = [(-0.0184, -0.0054), (-0.0065, 0.0065),
             (-0.013, 0.00005), (-0.0065, 0.0065)]
d_sum_prist = 0

for i5 in range(len(XX5g_prist)):
    d = dolya(XX5g_prist[i5][0], XX5g_prist[i5][1], YY5g_prist[i5][0], YY5g_prist[i5][1])
    d_sum_prist += 2*d
m_g5_prist = m_fg_prist * d_sum_prist
m_g5 = m_g5_prist + m_g5_yadr
k_m5 = m_ok5 / m_g5
print(k_m5)

3.23040645160536


#### Площадка № 6 (нижняя по пристеночному слою)
$K_{m6} = 2.43$

In [422]:
XX6ok_yadr = [(-0.0145, -0.0015)]
YY6ok_yadr = [0.012]
d_sum_yadr = 0

for i6 in range(len(XX6ok_yadr)):
    d = dolya_prist(XX6ok_yadr[i6][0], XX6ok_yadr[i6][1], YY6ok_yadr[i6])
    d_sum_yadr += 2*d

In [423]:
XX6ok_prist = [(-0.0125, 0.0005), (-0.0242, -0.0112)]
YY6ok_prist = [-0.0007, 0.0013]
d_sum_prist = 0

for i6 in range(len(XX6ok_prist)):
    d = dolya_prist(XX6ok_prist[i6][0], XX6ok_prist[i6][1], YY6ok_prist[i6])
    d_sum_prist += 2 * d
m_6ok = (d_sum_yadr * m_fok_yadr) + (d_sum_prist * m_fok_prist)

In [424]:
XX6g_yadr = [(-0.0065, 0.0065)]
YY6g_yadr = [0.0127]
d_sum_yadr = 0

for i6 in range(len(XX6g_yadr)):
    d = dolya_prist(XX6g_yadr[i6][0], XX6g_yadr[i6][1], YY6g_yadr[i6])
    d_sum_yadr += d


In [425]:
XX6g_prist = [(-0.0184, -0.0054), (-0.0065, 0.0065), (-0.0184, -0.0054),
             (-0.0257, -0.0127), (-0.0193, -0.0063), (-0.013, 0.00005),
             (-0.0065, 0.0065), (-0.0257, -0.127), (-0.0193, -0.0063), (-0.013, 0.00005)]
YY6g_prist = [0.00005, 0.001, 0.00005,
              0.004, 0.0054, -0.0062, 
              -0.0065, 0.004, 0.0054, -0.0062]
d_sum_prist = 0

for i6 in range(len(XX6g_prist)):
    d = dolya_prist(XX6g_prist[i6][0], XX6g_prist[i6][1], YY6g_prist[i6])
    d_sum_prist += d
m_6g = (d_sum_yadr * m_fg_yadr) + (d_sum_prist * m_fg_prist)
km_6 = m_6ok / m_6g

#### Площадка № 7 (средняя по пристеночному слою)
$K_{m7} = 2.92$

In [426]:
XX7ok_yadr = [(-0.0076, 0.0053), (0, 0.013)]
YY7ok_yadr = [0.0098, 0.0119]
d_sum_yadr = 0

for i7 in range(len(XX7ok_yadr)):
    d = dolya_prist(XX7ok_yadr[i7][0], XX7ok_yadr[i7][1], YY7ok_yadr[i7])
    d_sum_yadr += d
m_7ok_yadr = d_sum_yadr * m_fok_yadr

In [427]:
XX7ok_prist = [(-0.0065, 0.0065), (-0.0184, -0.0054), (-0.0184, -0.0054)]
YY7ok_prist = [0.001, 0.000005, 0.000005]
d_sum_prist = 0

for i7 in range(len(XX7ok_prist)):
    d = dolya_prist(XX7ok_prist[i7][0], XX7ok_prist[i7][1], YY7ok_prist[i7])
    d_sum_prist += d
m_7ok_prist = d_sum_prist * m_fok_prist
m_7ok = m_7ok_prist + m_7ok_yadr

In [428]:
XX7g_yadr = [(-0.0156, -0.0056)]
YY7g_yadr = [0.0078]
d_sum_yadr = 0

for i7 in range(len(XX7g_yadr)):
    d = dolya_prist(XX7g_yadr[i7][0], XX7g_yadr[i7][1], YY7g_yadr[i7])
    d_sum_yadr += d
m_7g_yadr = d_sum_yadr * m_fg_yadr

In [429]:
XX7g_prist = [(-0.0257, -0.0127), (-0.0193, -0.0063), (-0.013, 0.00005),
             (-0.0065, 0.0065), (-0.0257, -0.127), (-0.0193, -0.0063), (-0.013, 0.00005),
             (-0.0125, 0.0005), (-0.0242, -0.0112), (-0.0125, 0.0005), (-0.0242, -0.0112)]
YY7g_prist = [0.004, 0.0054, -0.0062, 
              -0.0065, 0.004, 0.0054, -0.0062,
             -0.0007, 0.0013, -0.0007, 0.0013]
d_sum_prist = 0

for i7 in range(len(XX7g_prist)):
    d = dolya_prist(XX7g_prist[i7][0], XX7g_prist[i7][1], YY7g_prist[i7])
    d_sum_prist += d
m_7g_prist = d_sum_prist * m_fg_prist
m_7g = m_7g_prist + m_7g_yadr
km_7 = m_7ok / m_7g

#### Площадка № 8 (верхняя по пристеночному слою)
$K_{m8} = 0.917$

In [430]:
XX8ok_yadr = [(-0.0065, 0.0065)]
YY8ok_yadr = [0.0115]
d_sum_yadr = 0

for i8 in range(len(XX8ok_yadr)):
    d = dolya_prist(XX8ok_yadr[i8][0], XX8ok_yadr[i8][1], YY8ok_yadr[i8])
    d_sum_yadr += d
m_8ok_yadr = d_sum_yadr * m_fok_yadr

In [431]:
XX8ok_prist = [(-0.0125, 0.0005), (-0.0242, -0.0112)]
YY8ok_prist = [-0.0007, 0.0013]
d_sum_prist = 0

for i8 in range(len(XX8ok_prist)):
    d = dolya_prist(XX8ok_prist[i8][0], XX8ok_prist[i8][1], YY8ok_prist[i8])
    d_sum_prist += 2 * d

m_8ok_prist = d_sum_prist * m_fok_prist
m_8ok = m_8ok_prist + m_8ok_yadr

In [432]:
XX8g_yadr = [(-0.0134, -0.0004)]
YY8g_yadr = [0.0075]
d_sum_yadr = 0

for i8 in range(len(XX8g_yadr)):
    d = dolya_prist(XX8g_yadr[i8][0], XX8g_yadr[i8][1], YY8g_yadr[i8])
    d_sum_yadr += 2*d
m_8g_yadr = d_sum_yadr * m_fg_yadr

In [433]:
XX8g_prist = [(-0.0184, -0.0054), (-0.0065, 0.0065), (-0.0184, -0.0054),
             (-0.0257, -0.0127), (-0.0193, -0.0063), (-0.013, 0.00005),
             (-0.0065, 0.0065), (-0.0257, -0.127), (-0.0193, -0.0063), (-0.013, 0.00005)]
YY8g_prist = [0.00005, 0.001, 0.00005,
              0.004, 0.0054, -0.0062, 
              -0.0065, 0.004, 0.0054, -0.0062]
d_sum_prist = 0

for i8 in range(len(XX8g_prist)):
    d = dolya_prist(XX8g_prist[i8][0], XX8g_prist[i8][1], YY8g_prist[i8])
    d_sum_prist += d
m_8g_prist = d_sum_prist * m_fg_prist
m_8g = m_8g_prist + m_8g_yadr
km_8 = m_8ok / m_8g

### Расчет центробежной форсунки окислителя в ядре

$\Delta p_ф = 2~МПа;~ \rho_{ок} = 1140 \frac{кг}{м^3};~ \eta_{ок} = 0.2 \cdot 10^{-3} Па \cdot с;~
\alpha = 45^{\circ}$



In [434]:
delta_p_f = 2 * 10 ** 6
ro_ok = 1140 
din_vyaz = 0.2 * (10 ** (-3))
alfa = 45

### Диаметр форсунки и толщина боковой стенки 
$D_ф = 6мм;~~ \delta_{ст}=1.5мм$

In [435]:
D_f = H / 2
delt = 0.0015

### Димаетр камеры закручивания, входного тангенциального отверстия и число тангенциальных отверстий
$D_{кз}=3мм;~~d_{вх}=1мм;~~i_{вх}=4$

In [436]:
D_kz = D_f - (2 * delt)
d_vx = 0.001
r_vx = d_vx / 2
i_vx = 4

### Длина тангенциального отверстия 

$l_{вх}= 1.71мм$

In [437]:
l_vx = ((((D_f/2)**2)-((((D_kz-d_vx)/2))**2))**0.5)-((((D_kz/2)**2)-((((D_kz-d_vx)/2))**2))**0.5)

### Длина камеры закручивания, принимаем ${l'}_{кз} = 1.2$ 
$\\ l_{кз}=3.6мм$


In [438]:
l_sht_kz = 1.2
l_kz = l_sht_kz * D_kz

### угол наклона образующей конической поверхности перехода от камеры закручивания к соплу 
$\phi_c=70^\circ$

In [439]:
fi_c = 70

### Величина радиуса, на котором расположена ось входного тангеницального отверстия от оси форсунке
$R_{вх} = 1.25мм$

In [440]:
R_vx = (D_kz-d_vx) / 2

### Диаметр сопла форсунки 
$d_c = 2.8мм$

In [441]:
d_c = 0.7*D_kz

### Относительная длина сопла ${l'}_c=0.8$
### Длина сопла
$l_c=1.7мм$

In [442]:
l_shc=0.8
l_c = l_shc*d_c

### Угол между осью входного тангенциального отверстия и наппавлением оси форсунки $\theta = 90^{\circ}$

### Геометрическая характеристика центробежной форсунки
$A=1.05$

In [443]:
theta = 90
A = ((R_vx*(d_c/2)) / (i_vx*((d_vx/2)**2))) * math.sin(3.14*theta/180)

### Число Рейнольдса на входе в форсунку 
$Re_{вх} = 4.05 \cdot 10^5$

In [444]:
Re = (4 * m_fok_yadr) / (math.pi * din_vyaz * d_vx * math.sqrt(i_vx))

### Коэффициент трения при условиях на входе в форсунку
$\lambda = 0.183$

In [445]:
lambd = math.exp(((25.8) / ((math.log10(Re)) ** 2.58)) - 2)

### Эквивалентная геометрическая характеристика форсунки, учитывающая влияние вязкости реальной жидкости
$A_э = 1.154$

In [446]:
A_e = A / (1 + ((lambd/2) * ((((R_vx / (r_vx * i_vx)) ** 2) * (1 / i_vx)) - A)))

### Коэффициент живого сечения сопла форсунки 
$\varphi = 0.610$

In [447]:
fi = 1/ (((((A_e / math.sqrt(8)) + math.sqrt(((A_e ** 2)/8) - (1/27))) ** (1/3)) + \
     (((A_e / math.sqrt(8)) - math.sqrt(((A_e ** 2)/8) - (1/27))) ** (1/3))) ** 2)

### Коэффициент расхода в центробежной форсунки
$\mu = 0.405$

In [448]:
mu_fi = fi * ((fi / (2 - fi)) ** 0.5)

### Длина форсунки 
$h = 6.5мм;$

In [449]:
h = l_kz + l_c + (((D_kz - d_c) / 2) * math.tan(math.pi * fi_c / 180))